# Human Activity Recognition

Database built from the recordings of 30 subjects performing activities of daily living while carrying a waist-mounted smartphone with embedded inertial sensors.

Each person performed six activities (WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING)

In [2]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/XOEN9W05_4A"' 
     'frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

Sources:

Project: https://github.com/guillaume-chevalier/LSTM-Human-Activity-Recognition

Data: https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

# Load Libraries

In [3]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical

Using TensorFlow backend.


# Load Data

Each axis of each signal is stored in a separate file, meaning that each of the train and test datasets have nine input files to load and one output file to load.

The input data is in CSV format where columns are separated by whitespace.  Each of these files can be loaded as a NumPy array. 

Function for loading the entire dataset:

In [63]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix):
    # load data and labels
    X, y = load_dataset_group(prefix)
    
    # zero-offset class values
    y = y - 1
    
    # one hot encode y
    y = to_categorical(y)
    
    # return dataset
    return X, y

A function for loading a dataset group of files

In [57]:
# load a dataset group, such as train or test
def load_dataset_group(group):
    
    # load all 9 files as a single array
    filenames = list()
    
    # total acceleration
    filenames += ['/Inertial Signals/total_acc_x.txt',
                  '/Inertial Signals/total_acc_y.txt',
                  '/Inertial Signals/total_acc_z.txt']
    
    # body acceleration
    filenames += ['/Inertial Signals/body_acc_x.txt',
                  '/Inertial Signals/body_acc_y.txt',
                  '/Inertial Signals/body_acc_z.txt']
    
    # body gyroscope
    filenames += ['/Inertial Signals/body_gyro_x.txt',
                  '/Inertial Signals/body_gyro_y.txt',
                  '/Inertial Signals/body_gyro_z.txt']
    
    # load input data
    X = load_group(filenames, group)
    
    # load class output
    y = load_file(group+'/Inertial Signals/y_labels.txt')
    
    # return X and y
    return X, y

A function for loading a group of files

In [58]:
# load a list of files and return as a 3d numpy array
def load_group(filenames, group):
    loaded = list()
    
    for name in filenames:

        data = load_file(group+name)
        loaded.append(data)
    
    # stack group so that features are the 3rd dimension
    loaded = dstack(loaded)
    return loaded

A function for loading a single file

In [59]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

Execute Function-Chain

In [68]:
# load training data
trainX, trainy = load_dataset('data_labelled_training')

In [69]:
# load test data
testX, testy = load_dataset('data_labelled_performance')

# Analyse Data

There are three main signal types in the raw data:
- total acceleration
- body acceleration
- body gyroscope

Each has three axes of data. This means that there are a total of __nine variables for each time step__. 

Further, each serie sof data has been partitioned into overlapping windows of 2.65 seconds of data, or 128 time steps.

These windows of data correspond to the windows of engineered features (rows) in the previous section.

This means that one row of data has (128×9), or 1,152, elements

In [70]:
len(trainX)

7352

In [71]:
len(testX)

2947

In [15]:
len(trainX[0])

128

# Train Model

Define and train model:

In [107]:
# fit and evaluate a model
def train_model(trainX, trainy):
     
    # define parameters
    verbose = 0
    epochs = 25
    batch_size = 64
    n_outputs = 6 # number of classes    
    time_steps = 4
    rows = 1
    columns = 32
    channels = 9 #number of features
    samples = trainX.shape[0]
    
    # define model
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(time_steps, rows, columns, channels)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # reshape data into subsequences (samples, time steps, rows, cols, channels)
    trainX = trainX.reshape((samples, time_steps, rows, columns, channels))
        
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)    

    return model

In [102]:
model = train_model(trainX, trainy)

# Evaluate Performance

In [103]:
def evaluate_performance(testX, testy):

    # reshape data into subsequences (samples, time steps, rows, cols, channels)
    samples = testX.shape[0]
    time_steps = 4
    rows = 1
    columns = 32
    channels = 9 #number of features    
    testX = testX.reshape((samples, time_steps, rows, columns, channels))
    
    loss, accuracy = model.evaluate(testX, testy, verbose=0)
    return loss, accuracy   

In [108]:
loss, accuracy = evaluate_performance(testX, testy)

In [109]:
loss, accuracy

(0.6875212619360295, 0.913810670375824)

# Store Performance

In [110]:
from datetime import datetime
# datetime object containing current date and time
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

Create Performance file:

In [115]:
import csv   
fields=['dt_string', 'loss', 'accuracy']
with open(r'model/performance.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(fields)

Add Performance Result:

In [116]:
import csv   
fields=[dt_string, loss, accuracy]
with open(r'model/performance.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(fields)

# Store Model for Deployment

Save model structure:

In [105]:
# serialize model to JSON
model_json = model.to_json()
with open("model/model_structure.json", "w") as json_file:
    json_file.write(model_json)

Save model weights:

In [106]:
# serialize weights to HDF5
model.save_weights("model/model_weights.h5")